## 1. Init

In [18]:
import os
import os.path as op
import nltk
import re
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
from collections import Counter

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from func import *

## 2. Feature Engineering
### 2.1 Blogs

In [ ]:
#Mine POS patterns
tagList = [
    'NN', 'CC', 'LS', 'PDT', 'POS', 'SYM', 'NNS', 'NNP', 'NNPS', 'FW', 'CD',
    'JJ', 'JJR', 'JJS', 'IN', 'TO', 'DT', 'EX', 'PRP', 'PRP$', 'WDT', 'WP',
    'WP$', 'MD', 'VB', 'VBZ', 'VBP', 'VBD', 'VBN', 'VBG', 'RB', 'RBR', 'RBS',
    'RP', 'WRB', 'UH', '.'
]

for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in range(0, len(blogs_gender)):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in os.listdir(txtDir + m):
            text = gettext(txtDir + m + '/' + file)
            sentences = nltk.sent_tokenize(text)
            CorpusPOS(sentences)

infile = open('CorpusPOS.txt', 'r')
cPOS = infile.readlines()
infile.close()
(a, b, c, d, e) = calc_probabilities(cPOS)
q1_output(a, b, c, d, e)

Prob = {}
infile = open('probabilities.txt', 'r')
prob_text = infile.readlines()

for sentence in prob_text:
    keyValPair = sentence.split(":")
    Prob[keyValPair[0]] = float(keyValPair[1][:-1])

infile.close()

posFeatures = minePOSPats(cPOS)

In [ ]:
#Feature extract
names = []
F_features = []
GRF_features = []
WC_features = []
labels = []

for gender in [0, 1]:
    if gender == 0:
        txtDir = './blogs-2000/female/'
    else:
        txtDir = './blogs-2000/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in tqdm(range(0, len(blogs_gender))):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in tqdm(os.listdir(txtDir + m), leave=False):
            name = txtDir + m + '/' + file
            text = gettext(name)
            words = nltk.word_tokenize(text)
            tags = nltk.pos_tag(words)
            words_l = wordlemmatize(tags)

            F_feature = F_measure(tags)
            GRF_feature = Gender_Preferential_Features(words_l)
            WC_feature = Word_Classes_Feature(words_l)

            names.append(name)
            lengths.append(length)
            F_features.append(F_feature)
            GRF_features.append(GRF_feature)
            WC_features.append(WC_feature)
            labels.append(gender)

WC_features_l = []
GRF_features_l = []

for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

map1 = {'name': names, 'label': labels, 'F_feature': F_features}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)

F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_blogs-2000.csv', index=False)

In [ ]:
#Get single
WC_features_l = []
for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

GRF_features_l = []
for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

POS_features_l = []
for i in range(len(POS_features[0])):
    n = i
    POS_features_l.append(getsingle(POS_features, n))

In [ ]:
#Get DataFrame
map1 = {'name': names, 'label': labels, 'F_feature': F_features}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

for i in range(len(POS_features[0])):
    key = 'POS_' + str(i + 1)
    value = POS_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)
F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_blogs.csv', index=False)
genderbias = pd.read_csv('blogs_genderbias.csv')
allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    'word ratio']
allofall.to_csv('blogs_features.csv', index=False)

In [12]:
#joint
for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
        outdir = './blogs-2000/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'
        outdir = './blogs-2000/male/'

    if not op.exists(outdir):
        os.makedirs(outdir)
    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in tqdm(range(0, len(blogs_gender))):
        m = blogs_gender[i]
        handle = ''
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in tqdm(os.listdir(txtDir + m),leave=False):
            name = txtDir + m + '/' + file
            text = gettext(name)
            handle = handle + ' ' + text
        os.mkdir(op.join(outdir, m))
        path = op.join(outdir, m, f'{m}.txt')
        with open(path, 'w') as w:
            w.write(handle)

Processing gender: ./postprocess_blogs/blogs/female/


Processing:  Tourism
Files: 54


Processing:  HumanResources
Files: 53


Processing:  RealEstate
Files: 39


Processing:  Arts
Files: 419


...

Processing:  Law
Files: 88


Processing:  Automotive
Files: 37


Processing:  Maritime
Files: 13


Processing:  Military
Files: 84


Processing:  Communications-Media
Files: 270


In [17]:
#cut
for gender in [0, 1]:
    if gender == 0:
        txtDir = './blogs-2000/female/'
    else:
        txtDir = './blogs-2000/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in tqdm(range(0, len(blogs_gender))):
        m = blogs_gender[i]
        print("Processing: ", m)
        filepath = op.join(txtDir, m, f'{m}.txt')
        text = gettext(filepath)
        words = nltk.word_tokenize(text)
        imax = len(words) // 2000
        for i in tqdm(range(imax),leave=False):
            handle = words[i * 2000:(i + 1) * 2000]
            handle = ' '.join(handle)
            savepath = op.join(txtDir, m, f'{m}_{i+1}.txt')
            with open(savepath, 'w') as w:
                w.write(handle)
    

Processing gender: ./blogs-2000/female/


Processing:  Tourism


Processing:  HumanResources


Processing:  RealEstate


Processing:  Arts


...

Processing:  Law


Processing:  Automotive


Processing:  Maritime


Processing:  Military


Processing:  Communications-Media


### 2.2 Novels

In [ ]:
#Word count and feature extract
names = []
F_features = []
GRF_features = []
WC_features = []
labels = []
lengths = []
for gender in [0, 1]:
    if gender == 0:
        txtDir = './5000/female/'
    else:
        txtDir = './5000/male/'

    print("Processing gender: {}".format(txtDir))
    handle_gender = os.listdir(txtDir)
    print("Files:", len(handle_gender))
    for m in handle_gender:
        name = txtDir + m
        text = gettext(name)
        words = nltk.word_tokenize(text)
        tags = nltk.pos_tag(words)
        words_l = wordlemmatize(tags)

        length = len(words)
        F_feature = F_measure(tags)
        GRF_feature = Gender_Preferential_Features(words_l)
        WC_feature = Word_Classes_Feature(words_l)

        names.append(name)
        lengths.append(length)
        F_features.append(F_feature)
        GRF_features.append(GRF_feature)
        WC_features.append(WC_feature)
        labels.append(gender)

WC_features_l = []
GRF_features_l = []

for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

map1 = {
    'name': names,
    'label': labels,
    'F_feature': F_features,
    'word count': lengths
}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)

F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_5000.csv', index=False)
genderbias = pd.read_csv('5000_genderbias.csv')
allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    'word ratio']
allofall.to_csv('5000_features.csv', index=False)

In [ ]:
allofall['word count'].describe()

In [3]:
#shorten text
#length_list = [200, 400, 600, 800, 1200, 1600, 2000, 27850]
length_list = [12000, 14000, 16000, 18000]

for l in length_list:
    print('\nPresent length:{}'.format(l))
    for gender in [0, 1]:
        if gender == 0:
            txtDir = './5000/female/'
            saveDir = f'./5000-{l}/female/'
        else:
            txtDir = './5000/male/'
            saveDir = f'./5000-{l}/male/'
        if not op.isdir(f'./5000-{l}'):
            os.mkdir(f'./5000-{l}')
        if not op.isdir(saveDir):
            os.mkdir(saveDir)
        print("Processing gender: {}".format(txtDir))
        handle_gender = os.listdir(txtDir)
        print("Files:", len(handle_gender))
        for m in tqdm(handle_gender):
            name = txtDir + m
            text = gettext(name)
            words = nltk.word_tokenize(text)
            if len(words) < (l + 1):
                continue
            words = words[:l]
            text = ''
            for word in words:
                text = text + word + ' '
            with open(op.join(saveDir, m), 'w') as f:
                f.write(text)

  0%|          | 0/2500 [00:00<?, ?it/s]


Present length:12000
Processing gender: ./5000/female/
Files: 2500


  0%|          | 0/2500 [00:00<?, ?it/s]

Processing gender: ./5000/male/
Files: 2500


  0%|          | 0/2500 [00:00<?, ?it/s]

...

  0%|          | 0/2500 [00:00<?, ?it/s]


Present length:18000
Processing gender: ./5000/female/
Files: 2500


  0%|          | 0/2500 [00:00<?, ?it/s]

Processing gender: ./5000/male/
Files: 2500


100%|██████████| 2500/2500 [19:16<00:00,  2.56it/s]


In [4]:
#feature extract
length_list = [12000, 14000, 16000, 18000]
for l in length_list:
    names = []
    F_features = []
    GRF_features = []
    WC_features = []
    labels = []
    print('\nPresent length:{}'.format(l))
    for gender in [0, 1]:
        if gender == 0:
            txtDir = f'./5000-{l}/female/'
        else:
            txtDir = f'./5000-{l}/male/'

        print("Processing gender: {}".format(txtDir))
        handle_gender = os.listdir(txtDir)
        print("Files:", len(handle_gender))
        for m in tqdm(handle_gender):
            name = txtDir + m
            text = gettext(name)
            words = nltk.word_tokenize(text)
            tags = nltk.pos_tag(words)
            words_l = wordlemmatize(tags)

            F_feature = F_measure(tags)
            GRF_feature = Gender_Preferential_Features(words_l)
            WC_feature = Word_Classes_Feature(words_l)

            names.append(name)
            F_features.append(F_feature)
            GRF_features.append(GRF_feature)
            WC_features.append(WC_feature)
            labels.append(gender)

    WC_features_l = []
    GRF_features_l = []

    for i in range(len(WC_features[0])):
        n = i
        WC_features_l.append(getsingle(WC_features, n))

    for i in range(len(GRF_features[0])):
        n = i
        GRF_features_l.append(getsingle(GRF_features, n))

    map1 = {'name': names, 'label': labels, 'F_feature': F_features}

    for i in range(len(WC_features[0])):
        key = 'WC_' + str(i + 1)
        value = WC_features_l[i]
        map1[key] = value

    for i in range(len(GRF_features[0])):
        key = 'GRF_' + str(i + 1)
        value = GRF_features_l[i]
        map1[key] = value

    allofall = pd.DataFrame(map1)

    F_features_u = np.array(F_features)
    F_features_u = (F_features_u -
                    np.mean(F_features_u)) / np.std(F_features_u)
    allofall['F_feature'] = F_features_u

    allofall.to_csv(f'allofall_5000-{l}.csv', index=False)
    #genderbias = pd.read_csv(f'5000-{l}_genderbias.csv')
    #allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias['word ratio']
    #allofall.to_csv(f'5000-{l}_features.csv',index = False)

  0%|          | 0/2251 [00:00<?, ?it/s]


Present length:12000
Processing gender: ./5000-12000/female/
Files: 2251


  0%|          | 0/2140 [00:00<?, ?it/s]

Processing gender: ./5000-12000/male/
Files: 2140


  0%|          | 0/2213 [00:00<?, ?it/s]

...

  0%|          | 0/2153 [00:00<?, ?it/s]


Present length:18000
Processing gender: ./5000-18000/female/
Files: 2153


  0%|          | 0/2003 [00:00<?, ?it/s]

Processing gender: ./5000-18000/male/
Files: 2003


100%|██████████| 2003/2003 [32:58<00:00,  1.05s/it]


## 3. Train

In [2]:
def train(clf, data, target):
    clf.fit(data.values, target.values.reshape(-1, ))
    scores_clf_cv = cross_val_score(rnd_clf,
                                    data.values,
                                    target.values.reshape(-1, ),
                                    cv=5)
    print(scores_clf_cv)
    print("Accuracy: %0.6f (+/- %0.3f)" %
          (scores_clf_cv.mean(), scores_clf_cv.std() * 2))
    return scores_clf_cv.mean(), clf.feature_importances_

In [47]:
#train
allf = []
allf_i = []
gb = []
gb_i = []
without = []
without_i = []
featurelist = []
length_list = [12000, 14000, 16000, 18000]
list1 = ['occu', 'emo', 'verb']

for l in length_list:
    allofall = pd.read_csv(f'allofall_5000-{l}.csv')
    genderbias = pd.read_csv(f'5000-{l}_genderbias.csv')
    allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
        'word ratio']

    for j in range(3):
        genderbias = pd.read_csv(f'5000-{l}_genderbias-{list1[j]}.csv')
        allofall[f'bias-{list1[j]}'], allofall[
            f'word ratio{list1[j]}'] = genderbias['bias'], genderbias[
                'word ratio']

    allofall.to_csv(f'5000-{l}_features.csv', index=False)
    data, target = allofall.drop(
        columns=['name', 'label']).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))
    data1 = data.iloc[:, :-6]
    data2 = data.iloc[:, :-8]
    data3 = data.iloc[:, -8:-6]
    rnd_clf = RandomForestClassifier(n_estimators=800,
                                     max_leaf_nodes=50,
                                     n_jobs=-1)

    print('\n Length:{}'.format(l))
    a, b = train(rnd_clf, data1, target)
    allf.append(a)
    allf_i.append([b])
    a, b = train(rnd_clf, data2, target)
    without.append(a)
    without_i.append([b])
    a, b = train(rnd_clf, data3, target)
    gb.append(a)
    gb_i.append([b])


 Length:12000
[0.74744027 0.75626424 0.78929385 0.75968109 0.78473804]
Accuracy: 0.767483 (+/- 0.033)
[0.71217292 0.72323462 0.75056948 0.7357631  0.74829157]
Accuracy: 0.734006 (+/- 0.029)
[0.70875995 0.68451025 0.70501139 0.70387244 0.72437358]
Accuracy: 0.705306 (+/- 0.025)

 Length:14000
[0.76860465 0.77093023 0.76046512 0.74825175 0.78088578]
Accuracy: 0.765828 (+/- 0.022)
[0.73372093 0.73372093 0.72325581 0.74125874 0.76107226]
Accuracy: 0.738606 (+/- 0.025)
[0.73837209 0.70116279 0.69069767 0.7004662  0.71095571]
Accuracy: 0.708331 (+/- 0.033)

 Length:16000
[0.78343195 0.78579882 0.75976331 0.78436019 0.74614472]
Accuracy: 0.771900 (+/- 0.032)
[0.73964497 0.7704142  0.73254438 0.74526066 0.72004745]
Accuracy: 0.741582 (+/- 0.033)
[0.71005917 0.72544379 0.67928994 0.72274882 0.6737841 ]
Accuracy: 0.702265 (+/- 0.043)

 Length:18000
[0.77163462 0.77283654 0.74639423 0.79277108 0.8060241 ]
Accuracy: 0.777932 (+/- 0.041)
[0.71875    0.75841346 0.72475962 0.77831325 0.75180723]
Acc

In [55]:
#length vs accuracy
df = pd.read_csv('length_v_acc_and_im.csv')
df = df.set_index('length')
allofall = pd.read_csv('5000-10000_features.csv')

allf_ii = [0] * len(length_list)
without_ii = [0] * len(length_list)
gb_ii = [0] * len(length_list)
name = allofall.columns[2:]
for i in range(len(length_list)):
    allf_ii[i] = sorted(list(zip(name, allf_i[i][0])),
                        key=lambda x: x[1],
                        reverse=True)
    without_ii[i] = sorted(list(zip(name[:-2], without_i[i][0])),
                           key=lambda x: x[1],
                           reverse=True)
    gb_ii[i] = sorted(list(zip(name[-2:], gb_i[i][0])),
                      key=lambda x: x[1],
                      reverse=True)

allf_iii = [0] * len(length_list)
without_iii = [0] * len(length_list)
name = ['F', 'WC', 'GRF', 'bias', 'word ratio']
for i in range(len(length_list)):
    temp = allf_i[i]
    handle = [0] * 5
    handle[0] = temp[0][0]
    handle[1] = sum(temp[0][1:24])
    handle[2] = sum(temp[0][24:34])
    handle[3:] = temp[0][-2:]
    handle = sorted(zip(name, handle), key=lambda x: x[1], reverse=True)
    allf_iii[i] = handle
    temp = without_i[i]
    handle = [0] * 3
    handle[0] = temp[0][0]
    handle[1] = sum(temp[0][1:24])
    handle[2] = sum(temp[0][24:34])
    handle = sorted(zip(name[:-2], handle), key=lambda x: x[1], reverse=True)
    without_iii[i] = handle

start = 8
df1 = df.iloc[:8, :]
df2 = df.iloc[:len(length_list),:]
df2.index = length_list
df3 = df.iloc[8:, :]
df = pd.concat([df1, df2, df3])
for i in range(len(length_list)):
    df.iloc[start + i, 0] = allf[i]
    df.iloc[start + i, 3] = without[i]
    df.iloc[start + i, 6] = gb[i]
    df.iloc[start + i, 1] = [allf_ii[i][:5]]
    df.iloc[start + i, 4] = [without_ii[i][:5]]
    df.iloc[start + i, 7] = [gb_ii[i]]
    df.iloc[start + i, 2] = [allf_iii[i]]
    df.iloc[start + i, 5] = [without_iii[i]]
#df.to_csv('top5_sum.csv')

## 4. Others

In [ ]:
#10wincv
from sklearn.model_selection import cross_val_score

for i in range(1, 11):
    fname = str(10 + i) + '_win_5000_genderbias.csv'
    genderbias = pd.read_csv(fname)
    allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
        'word ratio']
    data, target = allofall.drop(columns=['name', 'label', 'word count']
                                 ).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))

    rnd_clf = RandomForestClassifier(n_estimators=1000,
                                     max_leaf_nodes=50,
                                     n_jobs=-1)
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('Windows:%.2f'%(i))
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

In [2]:
#train
allf = []
allf_i = []
bias1 = []
bias1_i = []
bias2 = []
bias2_i = []
bias3 = []
bias3_i = []
list1 = ['occu', 'emo', 'verb']

for l in length_list:
    allofall = pd.read_csv(f'allofall_5000-{l}.csv')
    for j in range(3):
        genderbias = pd.read_csv(f'5000-{l}_genderbias-{list1[j]}.csv')
        allofall[f'bias-{list1[j]}'], allofall[
            f'word ratio{list1[j]}'] = genderbias['bias'], genderbias[
                'word ratio']
    allofall.to_csv(f'5000-{l}_features.csv', index=False)
    data, target = allofall.drop(
        columns=['name', 'label']).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))
    data1 = data.iloc[:, :-6]
    data2 = data.iloc[:, :-8]
    data3 = data.iloc[:, -8:-6]
    rnd_clf = RandomForestClassifier(n_estimators=800,
                                     max_leaf_nodes=50,
                                     n_jobs=-1)

    print('\n Length:{}'.format(l))
    a, b = train(rnd_clf, data1, target)
    allf.append(a)
    allf_i.append([b])
    a, b = train(rnd_clf, data2, target)
    bias1.append(scores_rnd_clf_cv.mean())
    bias1_i.append([rnd_clf.feature_importances_])
    a, b = train(rnd_clf, data3, target)
    bias2.append(scores_rnd_clf_cv.mean())
    bias2_i.append([rnd_clf.feature_importances_])

    bias3.append(scores_rnd_clf_cv.mean())
    bias3_i.append([rnd_clf.feature_importances_])


 Length:10000
All features:
[0.76       0.77444444 0.77419355 0.77196885 0.77085651]
Accuracy: 0.77 (+/- 0.01)
bias1:
[0.68888889 0.69888889 0.70967742 0.72302558 0.70189099]
Accuracy: 0.70 (+/- 0.02)
bias2:
[0.69       0.69333333 0.71746385 0.72747497 0.70967742]
Accuracy: 0.71 (+/- 0.03)
bias3:
[0.69       0.69888889 0.71301446 0.72636263 0.71301446]
Accuracy: 0.71 (+/- 0.03)

 Length:20000
All features:
[0.78694581 0.7770936  0.76724138 0.79433498 0.7654321 ]
Accuracy: 0.78 (+/- 0.02)
bias1:
[0.69581281 0.67980296 0.68842365 0.69334975 0.71111111]
Accuracy: 0.69 (+/- 0.02)
bias2:
[0.70935961 0.69458128 0.70320197 0.72044335 0.72592593]
Accuracy: 0.71 (+/- 0.02)
bias3:
[0.71182266 0.69704433 0.70566502 0.7229064  0.72345679]
Accuracy: 0.71 (+/- 0.02)


In [7]:
name = allofall.columns[2:]
new = []
for i in range(2):
    new.append(sorted(zip(name,allf_i[i][0]),key=lambda x:x[1],reverse=True))
allf_i = new

[[('word ratioverb', 0.09792318988264424),
  ('word ratioemo', 0.08803981272675876),
  ('word ratio', 0.08689822974440123),
  ('word ratiooccu', 0.08665056375966053),
  ('GRF_3', 0.08562365749490158),
  ('GRF_2', 0.05542545657451242),
  ('F_feature', 0.054956674238661136),
  ('WC_1', 0.02695101097843183),
  ('bias', 0.026811438743032955),
  ('GRF_8', 0.025263404929873924),
  ('WC_15', 0.023937535941254383),
  ('WC_2', 0.021372038041099616),
  ('WC_21', 0.019388978128335944),
  ('WC_17', 0.018955222906931236),
  ('WC_13', 0.01883953855069007),
  ('WC_4', 0.017872635275490805),
  ('WC_6', 0.016798315218526785),
  ('WC_23', 0.016631524092157006),
  ('WC_10', 0.013086739594023082),
  ('GRF_1', 0.01272754277044346),
  ('WC_22', 0.012467319962607536),
  ('WC_7', 0.012320246269774316),
  ('bias-verb', 0.012177769954146914),
  ('GRF_9', 0.011916296763889604),
  ('WC_5', 0.011740002949498216),
  ('WC_11', 0.011643502536574006),
  ('WC_12', 0.011410907742312492),
  ('GRF_7', 0.01048429320561239)

In [30]:
def train(clf, data, target):
    clf.fit(data.values, target.values.reshape(-1, ))
    scores_clf_cv = cross_val_score(rnd_clf,
                                    data.values,
                                    target.values.reshape(-1, ),
                                    cv=5)
    print(scores_clf_cv)
    print("Accuracy: %0.6f (+/- %0.3f)" %
          (scores_clf_cv.mean(), scores_clf_cv.std() * 2))
    return scores_clf_cv.mean(), [clf.feature_importances_]


test = pd.concat([data.iloc[:, :-6], data.iloc[:, -6:]], axis=1)
a, b = train(rnd_clf, test, target)

[0.78078818 0.77832512 0.7635468  0.79679803 0.76666667]
Accuracy: 0.777225 (+/- 0.024)


In [35]:
data.columns = allofall.columns[2:]
test = data.loc[:, new[1][0][0]]
for f in new[1][1:15]:
    test = pd.concat([test, data.loc[:, f[0]]],axis=1)
a, b = train(rnd_clf, test, target)

[0.78448276 0.76970443 0.75985222 0.7820197  0.76666667]
Accuracy: 0.772545 (+/- 0.019)


In [38]:
new[0].__len__()

42